In [1]:
#installing kaggle library
!pip install kaggle

# Uploading the kaggle.json texte

In [2]:
#configure the path
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


# Importing the twitter sentiments dataset

In [3]:
# API to fetch data from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 75% 61.0M/80.9M [00:00<00:00, 163MB/s]
100% 80.9M/80.9M [00:00<00:00, 162MB/s]


In [4]:
#unzip and extract the data
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('data extracted')


data extracted


# Importing dependencies

In [5]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
#printing the stopwords in english
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data processing

In [8]:
#loading the data from csv to pandas
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [9]:
#checking the size of data
twitter_data.shape

(1599999, 6)

In [10]:
#printing the first 5 rows
twitter_data.head(n=5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
# regarding the data sets we could remark that the columns names doesn't exist and the
# one existing there right now is just the first value of our data

In [12]:
#naming the columns manually and reading the data again
columns_name=['target','id','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1',names=columns_name)

In [13]:
#checking again the data labeled correct or not
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [14]:
#missing values's check
twitter_data.isnull().sum()
# ===> looking correct and no need to cleaning

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [15]:
#checking the distribution of target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

# Convert the target (label) from 4 to 1

In [16]:
twitter_data.replace({'target':{4:1}},inplace=True )

In [17]:
#checking the distribution of target columns
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [ ]:
#    0  ====> Negative Tweet
#    1  ====> Positive Tweet

# Stemming

stemming is the process of reducing a word to its root word

exp :
actor,actress,acting = act

In [18]:
#using porterstemer which we uploade
port_stem = PorterStemmer()

In [19]:
def stemming(content):
  #using re (regular expression)
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)  #move all caracters not alphabets from a to z
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split() #split all the words into a list
  stemmed_content =[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #trying to stem the content
  #stopwords is the library we uplloaded containing the words that doesn't add to much meaning
  stemmed_content=''.join(stemmed_content)
  return stemmed_content
#stemming is modifying the content we pass and remive to much 'bruit' that doesn't add too much meaning

In [ ]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

In [ ]:
print(twitter_data['stemmed_content'])

In [ ]:
print(twitter_data['target'])

In [ ]:
#we don't need date and username to know if the content is good or not
#separating data and label
X=twitter_data['stemmed_content'].values
y=twitter_data['target'].values

In [ ]:
print(X)

In [ ]:
print(y)

In [ ]:
#split data into train and test
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2, stratify=Y,random_state=2)
       #startify is for equal distribution of 0 and 1 (values of Y) between test and train


In [ ]:
print(X.shape,X_train.shape,X_test.shape)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
#converting the textual data to numerical data

vectorizer= TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test) #we don't use the fit because we don't want to affect the model parametres trained on the X_train

#this vectorizer will take each word in the X data and transform it to a numererical data giving it a value
#following its importance then the model will compare the importance of this value with the target (0 and 1)
#following these steps this is how the model is supposed to learn

# Training the machine learning model

**Logistic Regression**

In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,y_train)

**Model Evaluation**


In [ ]:
# accuracy score on the training data
X_train_predictions= model.predict(X_train)
training_data_accuracy = accuracy_score(y_train,X_train_predictions)

In [ ]:
print('Accuracy score on the training Data : ',training_data_accuracy)

In [ ]:
# accuracy score on the test data
X_test_predictions= model.predict(X_test)
test_data_accuracy = accuracy_score(y_test,X_test_predictions)

# Saving the trained Model

In [ ]:
import pickle

In [ ]:
#save the model into your project folder
filename ='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

Using the saved model for future predictions

In [ ]:
#loading the saved model
model=pickle.load(open('/content/trained_model.sav','rb')) #change the path of your model if its necessary

In [ ]:
X_new=X_test[200]  #200 random to see if the model is evaluating correctly
print(y_test[200])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')